<h1 style="color: #4285F4;">Cyclistic</h1>

<h3 style="color: #EA4335;">Bicicletas compartidas</h3>

<h2>Caso final del 
<span style="color: black; font-size: 1.5em;">Certificado</span>    
<span style="color: #4285F4; font-size: 1.5em;">G</span>
<span style="color: #EA4335; font-size: 1.5em;">o</span>
<span style="color: #FBBC05; font-size: 1.5em;">o</span>
<span style="color: #4285F4; font-size: 1.5em;">g</span>
<span style="color: #34A853; font-size: 1.5em;">l</span>
<span style="color: #EA4335; font-size: 1.5em;">e</span>
<span style="color: black; font-size: 1.0em;">:</span>
</h2>

<h1>Análisis computacional de datos</h1>

<h3><span style="color: #FBBC05;">por</span> <span style="color: black;">Joaquín Olea Ibarra</span></h3>
<br><br><br>

# Parte I. Introducción, Preparación y Limpieza de Datos

# --> TAREA
Cómo parte del equipo de marketing que actualmente diseña la estrategia de negocio de Cyclistic, se te ha pedido revisar los archivos con los registros de todos los viajes en bicicleta realizados en el 2022. 

Tu tarea es encontrar las diferencias entre los usuarios casuales y los usuarios con membresia. 

La información que reunas en tu análisis se utilizará como una de las fuentes de referencia en el diseño de la estrategia que lleva por objetivo convertir a los usuarios casuales en usuarios con membresía. 

# --> FUENTES
Cyclistic es una empresa ficticia. La información con la que se trabaja es facilitada por Divvy, un sistema de biciletas compartidas de la ciudad de Chicago, IL. 

El link 'https://divvy-tripdata.s3.amazonaws.com/index.html' que fue compartido, da acceso a los registros de viajes de bicicleta de los usuarios de divvy en la ciudad de Chicago. Los archivos están dividos en registros por mes y año, y se encuentran en formato zip. 

# --> ANTECEDENTES
Divvy cuenta con dos modelos de bicicletas: la clásica (classic) y la eléctrica (electric).

La bicicleta clásica tiene que liberarse y asegurarse de regreso en cualquiera de las estaciones que existen en la ciudad (sin necesidad de que sea la misma estación).

La electrica se libera abriendo un cable que será el que se utilize para rasegurarla de vuelata (se cierra el cable) algún poste o rack donde esté permitido hacerlo.

Actualmente se puede hacer uso de las bicicletas divvy como usuario casual o miembro:

--> Los usuarios 'casuales' pueden viajar en bicicleta divvy con estas modalidades: 

* Single Ride:
    * Classic bike: 1 dólar para liberar la bicicleta más 0.18 el minuto.
    * Electric bike: 1 dólar para liberar la bicicleta más 0.44 el minuto.
      
* Day Pass:
    * Classic bike: 18.10 por día, con viajes de 3 horas ilimitados y 0.18 el minuto extra.
    * Electric bike: 18.10 por día, sin cargos por liberación, y 0.44 el minuto. 

--> Los usuarios 'miembros' lo hacen bajo esta modalidad: 

* Membresía anual: 143.9 por año
    * Classic bike: con viajes sin costo de 45 minutos ilimitados y 0.18 el minuto extra.
    * Electric bike: sin cargos por liberación, y 0.18 el minuto. 

Regreso de las bicicletas: 
* Las bicicletas clásicas tienen que ser devueltas a una estación divvy.
* Las bicicletas eléctricas tienen que ser devueltas a una estación divvy o a una estación de e-bike.
* Las bicicletas elécticas pueden considerarse 'devueltas' si se aseguran a una rack o poste público por un costo extra: 1.1 para miembros y 2.2 para no miembros.
  
Nota: Esta información fue recopilda de 'https://divvybikes.com'


# --> ALCANCES

No se cuenta con información individual por usuario.

La relación entre el número de viajes y la duración de los viajes con las horas de sol, la temperatura, lluvia o nieve, estaría interesante (Se pudiera mapear las temperaturas por manzana contra los viajes realizados y el tipo de bicicleta en estaciones localizadas en dichas manzanas), pero no se aborda en este cuaderno. 




# 1. OBTENCIÓN DEL CONJUNTO DE DATOS A ANALIZAR        

                                                '''
**Objetivo:**

Nuestro propósito en esta etapa inicial es automatizar el proceso de adquisición de datos correspondientes al año 2022, facilitando así su análisis posterior. Este proceso incluye la descarga, lectura y exportación de archivos ZIP mensuales como archivos Parquet, almacenados en una carpeta designada para su acceso y manejo eficiente.

--> Fuente de Datos:
* Accedemos a un conjunto amplio de datos proporcionados por Divvy a través de su bucket de Amazon S3, disponible en [Divvy Tripdata](https://divvy-tripdata.s3.amazonaws.com/index.html). Los datos de interés para nuestro análisis del año 2022 están organizados y separados por mes, cada uno en un archivo ZIP individual.

--> Proceso de Descarga y Preparación:
Para lograr una gestión eficaz de estos datos, seguimos un proceso automatizado que incluye:

* Creación de Carpeta de Almacenamiento: Aseguramos un espacio organizado para los archivos Parquet generados, creando una carpeta divvy_tripdata si aún no existe.

* Descarga de Archivos ZIP: Mediante solicitudes HTTP, obtenemos los archivos ZIP mensuales directamente desde el bucket de S3. Cada archivo contiene los registros de viajes en bicicleta para un mes específico del año 2022.

* Extracción y Conversión de Datos: Extraemos el contenido CSV de cada archivo ZIP, lo leemos en un DataFrame de Pandas y posteriormente lo exportamos como archivo Parquet para optimizar el almacenamiento y la rapidez en consultas futuras. Este formato es particularmente adecuado para el manejo de grandes volúmenes de datos.

* Registro y Verificación: Mantenemos un registro de los archivos procesados y guardados, verificando la correcta ejecución del proceso. En caso de encontrar discrepancias, como archivos ZIP faltantes o problemas en la descarga, se registra el incidente para su revisión.

Al finalizar este proceso, contamos con una colección estructurada y accesible de datos de viajes en bicicleta para el año 2022, listos para ser analizados en las siguientes fases de nuestro proyecto.

In [1]:
import os
import pandas as pd
import requests
import zipfile
from io import BytesIO
import gc  # Garbage Collector

# Definir año para el que se descargarán los archivos
year = 2022

# Crear la carpeta si no existe
os.makedirs('csv_tripdata', exist_ok=True)

# Lista para almacenar los nombres de los archivos Parquet
parquet_files_list = []

for month in range(1, 13):
    # Formato del nombre del archivo
    file_name = f'{year}{month:02d}-divvy-tripdata'
    
    # URL del archivo ZIP
    zip_url = f'https://divvy-tripdata.s3.amazonaws.com/{file_name}.zip'

    # Descargar el archivo ZIP
    response = requests.get(zip_url)
    
    if response.status_code == 200:
        # Extraer el CSV del ZIP
        with zipfile.ZipFile(BytesIO(response.content)) as thezip:
            # Listar los nombres de los archivos contenidos en el ZIP
            file_names = thezip.namelist()

            # Filtrar y encontrar el nombre del archivo CSV, ignorando _MACOSX
            csv_file_name = next((name for name in file_names if name.endswith('.csv') and '_MACOSX' not in name), None)

            if csv_file_name:
                with thezip.open(csv_file_name) as csvfile:
                    # Leer el CSV en DataFrame
                    df = pd.read_csv(csvfile)
    
                    # Nombre del archivo Parquet
                    parquet_file = f'divvy_tripdata/{year}{month:02d}.parquet'
    
                    # Guardar el DataFrame en formato Parquet
                    df.to_parquet(parquet_file)
    
                    # Añadir el nombre del archivo a la lista
                    parquet_files_list.append(parquet_file)

                    # Información de diagnóstico
                    print(f"Archivo {parquet_file} procesado y guardado.")
    
                    # Limpiar el DataFrame para liberar memoria
                    del df
                    gc.collect()

            else:
                print(f"Archivo CSV no encontrado en el ZIP: {file_name}.zip")
    else:
        print(f"No se pudo descargar el ZIP para el mes {month:02d}.")

# Confirmar la finalización del proceso
print("Proceso completado")

Archivo divvy_tripdata/202201.parquet procesado y guardado.
Archivo divvy_tripdata/202202.parquet procesado y guardado.
Archivo divvy_tripdata/202203.parquet procesado y guardado.
Archivo divvy_tripdata/202204.parquet procesado y guardado.
Archivo divvy_tripdata/202205.parquet procesado y guardado.
Archivo divvy_tripdata/202206.parquet procesado y guardado.
Archivo divvy_tripdata/202207.parquet procesado y guardado.
Archivo divvy_tripdata/202208.parquet procesado y guardado.
Archivo divvy_tripdata/202209.parquet procesado y guardado.
Archivo divvy_tripdata/202210.parquet procesado y guardado.
Archivo divvy_tripdata/202211.parquet procesado y guardado.
Archivo divvy_tripdata/202212.parquet procesado y guardado.
Proceso completado


# 2. INFORMACIÓN BÁSICA Y PREPARACIÓN DE LOS DATOS

                                            '''

Antes de sumergirnos en el análisis, es crucial entender la estructura y el tipo de datos con los que trabajaremos. Para ello, utilizaremos nuestro propio módulo datatools, que incluye funciones como info y unique, diseñadas para ofrecer un vistazo rápido a nuestros datos y facilitar la toma de decisiones sobre el tratamiento adecuado de cada columna.

**Acciones:**

--> Análisis Preliminar:
* Utilizando la función info de datatools, realizamos un análisis preliminar para identificar el tipo de dato actual de cada columna y la presencia de valores nulos. La función unique, por otro lado, nos ayuda a comprender la diversidad de valores en columnas clave, facilitando la identificación de aquellas que se benefician de una conversión a tipos de datos más eficientes.

In [2]:
import pandas as pd
import datatools

enero = pd.read_parquet('divvy_tripdata/202201.parquet')

datatools.info(enero)

Pandas DataFrame
RangeIndex: 103770 entries, 0 to 103769
+-----+--------------------+--------+---------+---------+----------+
|   # | Column             |   Null | %Null   | Dtype   |   Unique |
+=====+====================+========+=========+=========+==========+
|   0 | ride_id            |      0 | 0.00%   | object  |   103770 |
+-----+--------------------+--------+---------+---------+----------+
|   1 | rideable_type      |      0 | 0.00%   | object  |        3 |
+-----+--------------------+--------+---------+---------+----------+
|   2 | started_at         |      0 | 0.00%   | object  |   100315 |
+-----+--------------------+--------+---------+---------+----------+
|   3 | ended_at           |      0 | 0.00%   | object  |   100047 |
+-----+--------------------+--------+---------+---------+----------+
|   4 | start_station_name |  16260 | 15.67%  | object  |      758 |
+-----+--------------------+--------+---------+---------+----------+
|   5 | start_station_id   |  16260 | 15.67%  

In [3]:
datatools.unique(enero)

--> Conteo para cada valor único de columna con valores únicos menores o iguales a 12

Columna: rideable_type
+----+---------------+---------+-------+
|    | Value         |   Count |     % |
+====+===============+=========+=======+
|  0 | classic_bike  |   55067 | 53.07 |
+----+---------------+---------+-------+
|  1 | electric_bike |   47742 | 46.01 |
+----+---------------+---------+-------+
|  2 | docked_bike   |     961 |  0.93 |
+----+---------------+---------+-------+

Columna: member_casual
+----+---------+---------+-------+
|    | Value   |   Count |     % |
+====+=========+=========+=======+
|  0 | member  |   85250 | 82.15 |
+----+---------+---------+-------+
|  1 | casual  |   18520 | 17.85 |
+----+---------+---------+-------+


--> Cómo lucen las primeras 5 filas: 

In [4]:
enero.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member


**--> Definición de Tipos de Datos Deseados:**

* Basándonos en nuestro análisis preliminar, definimos un diccionario desired_dtypes para estandarizar y optimizar los tipos de datos de nuestro DataFrame. La selección de tipos de datos se justifica en la naturaleza de los datos y el objetivo de nuestro análisis:

In [57]:
# Definir tipos de datos deseados para cada columna

desired_dtypes = {
    'ride_id': 'string',  # Preserva cualquier cero inicial y asegura compatibilidad universal.
    'rideable_type': 'category',  # Optimiza la memoria al limitar los valores a un conjunto finito.
    'started_at': 'datetime64[ns]',  # Facilita el análisis temporal con funciones específicas de tiempo.
    'ended_at': 'datetime64[ns]',
    'start_station_name': 'string',  # Acomoda datos textuales con variabilidad.
    'start_station_id': 'string',
    'end_station_name': 'string',
    'end_station_id': 'string',
    'start_lat': 'float64',  # Adecuado para datos geográficos, permitiendo cálculos precisos.
    'start_lng': 'float64',
    'end_lat': 'float64',
    'end_lng': 'float64',
    'member_casual': 'category'  # Reduce el uso de memoria y mejora el rendimiento de filtrado.
}


# 3. VERIFICIACIÓN DE CONSISTENCIA EN DATOS Y COLUMNAS; CONVERSIÓN DE DTYPES Y CONCATENACIÓN DE ARCHIVOS

                                            '''

El propósito de esta etapa es consolidar en un único conjunto de datos la información de viajes de bicicletas de cada mes del año para su posterior análisis. Sin embargo, antes de proceder con la concatenación, es esencial asegurar la uniformidad en la estructura de los datos y su compatibilidad con los tipos de datos previamente definidos.

**Proceso Detallado:**

--> Verificación Preliminar de la Estructura de Datos:

* Dado el volumen considerable de los archivos, optamos por una estrategia eficiente que implica verificar inicialmente solo una muestra de cada archivo Parquet. Esta muestra consiste en el primer grupo de filas, permitiendo una revisión rápida de las columnas sin la necesidad de cargar el archivo completo en memoria. Utilizamos pyarrow para esta tarea, aprovechando su capacidad para leer datos de manera eficiente.
Conversión de Tipos de Datos:

* Con la estructura de columnas confirmada, el siguiente paso es asegurar que los datos se puedan convertir correctamente a los tipos definidos anteriormente. Esta conversión se realiza en dos fases: primero, en una muestra de datos para verificar la viabilidad de la conversión; y luego, aplicando la conversión a todo el archivo al leerlo con pandas.

--> Lectura Completa y Conversión de Datos:

* Una vez validada la estructura de columnas y la conversión de tipos de datos en la muestra, procedemos a la lectura completa de cada archivo Parquet. Para esta operación, utilizamos pandas, aprovechando su funcionalidad para aplicar directamente los tipos de datos deseados durante la carga de los datos.

--> Concatenación de Datos Mensuales:

* Con cada archivo mensual de datos de viajes ahora en el formato deseado, finalizamos el proceso concatenando todos estos DataFrames en un único DataFrame para análisis. Esta concatenación se gestiona cuidadosamente para mantener la integridad de los datos y asegurar la coherencia a lo largo del conjunto de datos consolidado.

--> Ejecución y Registro de Errores:

* Implementamos un registro de errores para documentar cualquier incongruencia durante la verificación de columnas o la conversión de tipos de datos. Este enfoque nos permite abordar de manera proactiva cualquier problema, garantizando la calidad y consistencia de nuestros datos antes de la concatenación final.

In [6]:
import pyarrow.parquet as pq

# Leer el archivo de referencia completo
reference_file = pd.read_parquet('divvy_tripdata/202201.parquet')

# Aplicar la conversión de tipos de datos al archivo de referencia
reference_file = reference_file.astype(desired_dtypes)

# Guardar las columnas y tipos de datos del archivo de referencia para comparar más adelante
reference_columns = reference_file.columns
reference_dtypes = reference_file.dtypes

# Lista para almacenar los DataFrames de cada archivo Parquet
dataframes_list = []

# Abrir archivo para registro de errores
error_log = open("error_log.txt", "w")

# Iterar sobre cada archivo Parquet en la lista generada en la sección 1
for file in parquet_files_list:  
    try:
        # Leer el primer grupo de filas usando pyarrow
        parquet_file = pq.ParquetFile(file)
        table = parquet_file.read_row_group(0, columns=desired_dtypes.keys())
        sample_data = table.to_pandas()

        # Verificar si las columnas coinciden con el archivo de referencia
        if not sample_data.columns.equals(reference_columns):
            error_log.write(f"Las columnas del archivo {file} no coinciden con el archivo de referencia.\n")
            continue

        # Intentar aplicar la conversión de tipos de datos a la muestra
        try:
            sample_converted = sample_data.astype(desired_dtypes)
        except Exception as e:
            error_log.write(f"Error en la conversión de tipos de datos en {file}: {e}\n")
            continue
            
        # Leer el archivo completo y aplicar la conversión de tipos de datos
        monthly_data = pd.read_parquet(file).astype(desired_dtypes)

        # Añadir el DataFrame a la lista
        dataframes_list.append(monthly_data)

    except Exception as e:
        error_log.write(f"Error procesando el archivo {file}: {e}\n")

    # Limpiar la memoria
    gc.collect()

# Cerrar el archivo de registro de errores
error_log.close()

# Concatenar todos los DataFrames
trips = pd.concat(dataframes_list)

# 4. CAMBIO DE NOMBRE EN COLUMNAS

                                               '''

Para aumentar la claridad y consistencia en nuestro conjunto de datos, hemos identificado la necesidad de renombrar ciertas columnas. Esta decisión se basa tanto en la precisión semántica como en la uniformidad con las convenciones de nombramiento en análisis de datos.

**Renombramientos Específicos:**

--> Clarificación de Columnas de Fecha y Hora:

* Las columnas originalmente nombradas como 'started_at' y 'ended_at' podrían interpretarse ambiguamente como referencias a ubicaciones o a momentos temporales. Para evitar esta confusión y mejorar la legibilidad, hemos decidido cambiar sus nombres a 'start_datetime' y 'end_datetime', respectivamente. Este cambio refleja de manera más precisa que estas columnas contienen información de fechas y horas, facilitando su interpretación y manipulación en análisis temporales.

--> Uniformidad en la Identificación de Viajes y Tipos de Usuarios:

* Además, hemos optado por renombrar otras columnas para alinearlas con una terminología más intuitiva y coherente dentro del contexto de nuestro análisis:

    'ride_id' a 'trip_id': Este cambio enfatiza que cada registro representa un viaje individual.
  
    'rideable_type' a 'bike_type': Clarifica que la columna se refiere al tipo de bicicleta utilizada en el viaje.
  
    'member_casual' a 'user_type': Distingue de manera más clara entre usuarios con membresía y usuarios casuales.

--> Implementación del Cambio de Nombres:

* El siguiente código realiza los cambios de nombre mencionados, aplicándolos directamente al DataFrame trips:

In [7]:
trips.rename(columns={'ride_id': 'trip_id', 'rideable_type': 'bike_type', 'started_at': 'start_datetime', 
'ended_at': 'end_datetime', 'member_casual': 'user_type'}, inplace=True)

**Justificación de los Cambios:**

Aunque algunos de estos cambios podrían parecer menores o influenciados por preferencias personales, creemos firmemente que contribuyen a una mayor claridad y cohesión en nuestro conjunto de datos. Al adoptar nombres de columnas que reflejan de manera directa su contenido y propósito, facilitamos el análisis posterior y la comunicación de nuestros hallazgos.

# 5. CREACIÓN DE COLUMNAS 

                                                '''                   
Para enriquecer nuestro análisis y facilitar la extracción de insights, hemos decidido añadir varias columnas calculadas al conjunto de datos. Estas columnas proporcionarán información valiosa sobre la duración y temporalidad de los viajes en bicicleta.

**Columnas Añadidas:**

--> Duración del Viaje:

* Minutos (trip_minutes): Calculamos la duración de cada viaje en minutos. Esta métrica nos permite analizar la duración típica de los viajes y comparar patrones de uso entre diferentes segmentos de usuarios.

* Horas (trip_hours): Aunque similar a trip_minutes, calcular la duración en horas ofrece una perspectiva distinta, útil para identificar viajes largos y su distribución a lo largo del tiempo.

In [ ]:
# Calcular la duración del viaje en minutos
trips['trip_minutes'] = (trips['end_datetime'] - trips['start_datetime']).dt.total_seconds() / 60

# Calcualr la duración del viaje en horas
trips['trip_hours'] = (trips['end_datetime'] - trips['start_datetime']).dt.total_seconds() / (60 * 60)

--> Temporalidad del Viaje:

* Día de la Semana (week_day): Indica el día de la semana en que se realizó el viaje, basándonos en la fecha de inicio. Esta información es crucial para entender los patrones de uso a lo largo de la semana.

* Mes del Año (month): Revela el mes en que se efectuó el viaje. Analizar la distribución mensual de los viajes nos ayuda a identificar tendencias estacionales y el impacto de factores climáticos en el uso de bicicletas.

In [8]:
# Extraer el día de la semana
trips['week_day'] = trips['start_datetime'].dt.day_name()

# Convertir la columna 'week_day' a tipo de dato categórico
trips['week_day'] = trips['week_day'].astype('category')

# Extraer el mes del año
trips['month'] = trips['start_datetime'].dt.month_name()

# Convertir la columna 'month' a tipo de dato categórico
trips['month'] = trips['month'].astype('category')

**Justificación y Aplicación:**

* Estas columnas adicionales nos permiten realizar un análisis más granular y contextualizado de los datos de viaje. Por ejemplo, al examinar la duración del viaje en diferentes unidades (minutos y horas), podemos adaptar nuestro análisis a diversas preguntas de investigación, desde explorar la frecuencia de viajes cortos hasta identificar patrones en viajes excepcionalmente largos.

* Asimismo, las columnas relacionadas con el día de la semana y el mes del año abren la puerta a análisis temporales detallados, permitiéndonos investigar cómo varían los patrones de uso a lo largo del tiempo y en respuesta a factores externos.

# 6. INFORMACIÓN ESTADÍSTICA BÁSICA DEL DATAFRAME CON TODOS LOS ARCHIVOS CONCATENADOS

In [9]:
trips.describe()

,start_datetime,end_datetime,start_lat,start_lng,end_lat,end_lng,trip_minutes,trip_hours
count,5667717,5667717,5.667717e+06,5.667717e+06,5.661859e+06,5.661859e+06,5.667717e+06,5.667717e+06
mean,2022-07-20 07:21:18.741496832,2022-07-20 07:40:45.338642176,4.190222e+01,-8.764783e+01,4.190242e+01,-8.764790e+01,1.944329e+01,3.240548e-01
min,2022-01-01 00:00:05,2022-01-01 00:01:48,4.164000e+01,-8.784000e+01,0.000000e+00,-8.814000e+01,-1.035335e+04,-1.725558e+02
25%,2022-05-28 19:21:05,2022-05-28 19:43:07,4.188103e+01,-8.766154e+01,4.188103e+01,-8.766260e+01,5.816667e+00,9.694444e-02
50%,2022-07-22 15:03:59,2022-07-22 15:24:44,4.190000e+01,-8.764410e+01,4.190000e+01,-8.764414e+01,1.028333e+01,1.713889e-01
75%,2022-09-16 07:21:29,2022-09-16 07:39:03,4.193000e+01,-8.762957e+01,4.193000e+01,-8.762963e+01,1.846667e+01,3.077778e-01
max,2022-12-31 23:59:26,2023-01-02 04:56:45,4.563503e+01,-7.379648e+01,4.237000e+01,0.000000e+00,4.138725e+04,6.897875e+02
std,NaN,NaN,4.626109e-02,2.999925e-02,6.805821e-02,1.082985e-01,1.761281e+02,2.935469e+00


**OBSERVACIONES 1:** 
* Hay uno o varios valores negativos en las columnas de duración del viaje 'trip_minutes' y 'trip_hours'

**-->** Veamos cómo lucen: 

In [10]:
duracion_negativa = trips[trips['trip_minutes'] < 0]
duracion_negativa.head()

,trip_id,bike_type,start_datetime,end_datetime,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,user_type,trip_minutes,trip_hours,week_day,month
184212,2D97E3C98E165D80,classic_bike,2022-03-05 11:00:57,2022-03-05 10:55:01,DuSable Lake Shore Dr & Wellington Ave,TA1307000041,DuSable Lake Shore Dr & Wellington Ave,TA1307000041,41.936688,-87.636829,41.936688,-87.636829,casual,-5.933333,-0.098889,Saturday,March
187389,7407049C5D89A13D,electric_bike,2022-03-05 11:38:04,2022-03-05 11:37:57,Sheffield Ave & Wellington Ave,TA1307000052,Sheffield Ave & Wellington Ave,TA1307000052,41.936313,-87.652522,41.936253,-87.652662,casual,-0.116667,-0.001944,Saturday,March
592385,0793C9208A64302A,electric_bike,2022-05-30 11:06:29,2022-05-30 11:06:17,Broadway & Waveland Ave,13325,<NA>,<NA>,41.949069,-87.648499,41.950000,-87.650000,casual,-0.200000,-0.003333,Monday,May
126109,B897BE02B21FA75E,electric_bike,2022-06-07 19:15:39,2022-06-07 17:05:37,<NA>,<NA>,Kostner Ave & North Ave,519,41.900000,-87.740000,41.910000,-87.740000,casual,-130.033333,-2.167222,Tuesday,June
126532,072E947E156D142D,electric_bike,2022-06-07 19:14:46,2022-06-07 17:07:45,W Armitage Ave & N Sheffield Ave,20254.0,W Armitage Ave & N Sheffield Ave,20254.0,41.920000,-87.650000,41.920000,-87.650000,casual,-127.016667,-2.116944,Tuesday,June


**OBSERVACIONES 1.1:**
* Parecen estar invertidas las columnas start_datetime y end_datetime

**-->** Veamos cuántos son: 

In [11]:
duracion_negativa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 184212 to 330409
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   trip_id             100 non-null    string        
 1   bike_type           100 non-null    category      
 2   start_datetime      100 non-null    datetime64[ns]
 3   end_datetime        100 non-null    datetime64[ns]
 4   start_station_name  77 non-null     string        
 5   start_station_id    77 non-null     string        
 6   end_station_name    84 non-null     string        
 7   end_station_id      84 non-null     string        
 8   start_lat           100 non-null    float64       
 9   start_lng           100 non-null    float64       
 10  end_lat             100 non-null    float64       
 11  end_lng             100 non-null    float64       
 12  user_type           100 non-null    category      
 13  trip_minutes        100 non-null    float64    

**OBSERVACIONES 1.2:**

* Dada la pequeña proporción de registros afectados (100 de más de 5 millones) y la incertidumbre sobre la exactitud de otros campos en estos registros, se optó por eliminarlos para preservar la integridad del análisis general.

* La eliminación de estos registros mejora la calidad del conjunto de datos para análisis futuros, incluyendo estudios de duración promedio de viajes y patrones de uso por tipo de usuario. Sin embargo, es crucial implementar verificaciones adicionales de calidad para identificar y corregir posibles inconsistencias en datos geográficos y de estaciones.

* Además de los valores negativos en 'trip_minutes' y 'trip_hours', sería valioso examinar los extremos positivos de estas distribuciones, especialmente los viajes extremadamente largos que podrían influir en el análisis promedio y desviación estándar. Los valores máximos indican viajes de más de 40,000 minutos, lo cual es atípico y podría distorsionar el análisis. Pero esto se hará en la fase de análisis. 

**ACCIONES:**

**-->** Se eliminan registros con duraciones negativas.  

In [12]:
trips = trips[trips['trip_minutes'] >= 0].copy()

In [13]:
datatools.info(trips)

Pandas DataFrame
RangeIndex: 5667617 entries, 0 to 5667616
+-----+--------------------+--------+---------+----------------+----------+
|   # | Column             |   Null | %Null   | Dtype          |   Unique |
+=====+====================+========+=========+================+==========+
|   0 | trip_id            |      0 | 0.00%   | string         |  5667617 |
+-----+--------------------+--------+---------+----------------+----------+
|   1 | bike_type          |      0 | 0.00%   | category       |        3 |
+-----+--------------------+--------+---------+----------------+----------+
|   2 | start_datetime     |      0 | 0.00%   | datetime64[ns] |  4745787 |
+-----+--------------------+--------+---------+----------------+----------+
|   3 | end_datetime       |      0 | 0.00%   | datetime64[ns] |  4758572 |
+-----+--------------------+--------+---------+----------------+----------+
|   4 | start_station_name | 833041 | 14.70%  | string         |     1674 |
+-----+--------------------+-

**OBSERVACIONES 2 (sobre Datos Nulos en Estaciones y Coordenadas):**

--> Hallazgos Clave:

* Presencia Significativa de Valores Nulos: Existen valores nulos en las columnas relacionadas tanto con los nombres (station_name) como con los identificadores de estaciones (station_id), afectando tanto a las estaciones de inicio (start) como de destino (end). Este fenómeno es particularmente notable, con aproximadamente el 15% de los registros careciendo de esta información.

* Discrepancia en Datos de Coordenadas: Además, se identificó una menor proporción de valores nulos en las coordenadas de destino (end_lat y end_lng), representando solo el 0.10% del total de registros. Esta diferencia sugiere una distribución desigual de datos incompletos entre las diferentes columnas del conjunto de datos.

* Específico de Bicicletas Eléctricas: Dado que las bicicletas eléctricas ofrecen la flexibilidad de ser aseguradas en ubicaciones más variadas, como postes públicos o racks (además de en las estaciones oficiales), se anticipa naturalmente una mayor incidencia de registros nulos en las columnas station_name y station_id para este tipo de bicicleta.

--> Análisis por Tipo de Bicicleta:

Para comprender mejor esta tendencia, se procedió a agrupar los registros con valores nulos en función del tipo de bicicleta (bike_type). Este enfoque busca verificar si la prevalencia de registros nulos en las columnas de estaciones se correlaciona significativamente con el uso de bicicletas eléctricas, en comparación con las bicicletas clásicas o docked.

In [14]:
null_types = trips.groupby('bike_type', observed=False).agg(
    null_start_station=pd.NamedAgg(column='start_station_name', aggfunc=lambda x: x.isnull().sum()),
    null_end_station=pd.NamedAgg(column='end_station_name', aggfunc=lambda x: x.isnull().sum()),
    null_end_coordinates=pd.NamedAgg(column='end_lat', aggfunc=lambda x: x.isnull().sum())
)
null_types

,null_start_station,null_end_station,null_end_coordinates
bike_type,,,
classic_bike,0,3788,3242
docked_bike,0,2616,2616
electric_bike,833041,886322,0


**OBSERVACIONES 2.1 (Análisis Detallado de Datos Nulos en Estaciones y Coordenadas):**

--> Confirmación de Patrones por Tipo de Bicicleta:

* Bicicletas Eléctricas y Datos Nulos: Conforme a nuestras predicciones, los registros que carecen de información en la columna start_station_name pertenecen predominantemente a bicicletas eléctricas. Esto respalda la noción de que las bicicletas eléctricas, dada su capacidad para ser aseguradas en una gama más amplia de ubicaciones, generan más registros sin asignación a estaciones específicas.

* Bicicletas Clásicas y Docked – Posibles Bicicletas Extraviadas: Observamos que los datos nulos en end_station_name para bicicletas clásicas y docked podrían indicar situaciones donde las bicicletas han sido extraviadas o no se han retornado correctamente. Es intrigante notar que, de los 3788 registros sin estación final para bicicletas clásicas, 3242 carecen también de coordenadas finales, dejando una discrepancia de 546 registros. Esta diferencia podría sugerir que algunas bicicletas inicialmente considieradas como no devueltas quizás fueron encontradas o devueltas posteriormente a una estación.

--> Acción Inmediata:

* Mantendremos estos registros en el análisis por el momento, reservando una decisión sobre su tratamiento para una etapa posterior, tras una evaluación más exhaustiva.

--> Categorización y Uso del Término 'Docked':

* Clarificación de Categorías de Bicicletas: Se identificaron tres categorías de bicicletas en el conjunto de datos: classic, electric y docked. Sin embargo, la documentación oficial de Divvy menciona solo dos tipos: classic y electric. Esto sugiere que el término 'docked' podría haber sido empleado para describir bicicletas clásicas en contextos específicos, posiblemente para distinguir entre bicicletas que requieren ser devueltas a una estación de acoplamiento y aquellas eléctricas que ofrecen mayor flexibilidad de estacionamiento.

--> Investigación Adicional:

* Para profundizar en esta clasificación y comprender mejor el uso del término 'docked', hemos analizado los datos del primer cuatrimestre de 2020. Este análisis adicional tiene como objetivo verificar si 'docked_bike' se refiere específicamente a bicicletas clásicas que deben ser estacionadas en puertos de estaciones Divvy, contrastando así con las bicicletas eléctricas que permiten un estacionamiento más libre.

In [15]:
df = pd.read_csv('Divvy_Trips_2020_Q1.csv')

for col in df.columns:
        unique_count = df[col].nunique()  # Calcular el número de valores únicos para cada columna
        if unique_count <= 3 and unique_count > 0:
            print(f"\nColumna: {col}")
            
            # Obtener conteos y porcentajes
            value_counts = df[col].value_counts()
            percentages = df[col].value_counts(normalize=True) * 100

            # Crear DataFrame con Value, Count, y Percentage
            data = pd.DataFrame({
                'Value': value_counts.index,
                'Count': value_counts.values,
                '%': percentages.values
            })
            print(data)


Columna: rideable_type
         Value   Count      %
0  docked_bike  426887  100.0

Columna: member_casual
    Value   Count          %
0  member  378407  88.643365
1  casual   48480  11.356635


**OBSERVACIONES 2.2.1 (Confirmación y Ajuste de Categorías de Bicicletas):**

--> Observaciones Detalladas:

* Clarificación de la Categoría 'Docked_bike': Tras una revisión exhaustiva de los datos y comparación con información adicional, hemos confirmado que la categoría 'docked_bike' se utilizaba anteriormente para referirse a las bicicletas de tipo 'classic_bike'. Esta distinción es crucial para una interpretación precisa del conjunto de datos y para el análisis del comportamiento de uso de las bicicletas.

--> Acciones Implementadas:

* Estandarización de Categorías de Bicicletas: Para asegurar coherencia y claridad en nuestro conjunto de datos, hemos procedido a actualizar los registros etiquetados como 'docked_bike', reemplazándolos por la categoría 'classic_bike'. Este cambio armoniza la clasificación de las bicicletas, alineándola con la terminología oficial y facilitando análisis futuros.

In [16]:
# Reemplazar 'docked_bike' con 'classic_bike'
trips['bike_type'] = trips['bike_type'].replace('docked_bike', 'classic_bike')

# Verificar el tipo de datos y las categorías actuales
print(trips['bike_type'].dtype)
print(trips['bike_type'].cat.categories)

category
Index(['classic_bike', 'electric_bike'], dtype='object')


**OBSERVACIONES 3 (Análisis de la Diversidad de Ubicaciones de Inicio y Fin para Bicicletas Eléctricas):**

--> Observaciones Iniciales sobre Diversidad de Coordenadas:

* Alta Variabilidad en Coordenadas de Inicio: Hemos notado una considerable cantidad de valores únicos en las coordenadas de inicio ('start_lat' y 'start_lng'), superando los 600,000 valores distintos. Este fenómeno se destaca particularmente en el contexto de las bicicletas eléctricas, las cuales, gracias a su flexibilidad de estacionamiento, pueden ser aseguradas en una amplia variedad de ubicaciones, como postes o racks públicos.

--> Investigación Focalizada en Bicicletas Eléctricas:

* Para profundizar en este patrón, hemos decidido examinar cuántas de estas coordenadas únicas corresponden específicamente a bicicletas clasificadas bajo la categoría 'electric_bike'.

In [43]:
# DataFrames filtrados por tipo de bicicleta
electric = trips[trips['bike_type'] == 'electric_bike']
classic = trips[trips['bike_type'] == 'classic_bike']
start_coords = ['start_lat', 'start_lng', 'end_lat', 'end_lng']

# Lista para almacenar los resultados
results_data = []

# Calcular los valores únicos para cada tipo de bicicleta y cada coordenada
for bike_type, df in [('Electric Bike', electric), ('Classic Bike', classic), ('All Bikes', trips)]:
    unique_counts = {coord: df[coord].nunique() for coord in start_coords}
    unique_counts['Bike Type'] = bike_type
    results_data.append(unique_counts)

# Crear un DataFrame con los resultados
results_df = pd.DataFrame(results_data)

# Reordenar las columnas si es necesario
results_df = results_df[['Bike Type', 'start_lat', 'start_lng', 'end_lat', 'end_lng']]

# Mostrar los resultados
print("Valores únicos por tipo de bicicleta para columnas de coordenadas")
print()
print(results_df)


Valores únicos por tipo de bicicleta para columnas de coordenadas

       Bike Type  start_lat  start_lng  end_lat  end_lng
0  Electric Bike     667669     630402     1282     1267
1   Classic Bike       1146       1145     1096     1077
2      All Bikes     668336     631033     1606     1590


In [41]:
electri_classic_common_unique_values = {}
columnas = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
for col in columnas: 
    unique_col_classic = set(classic[col])
    unique_col_electric = set(electric[col])
    common_col_values = unique_col_classic.intersection(unique_col_electric)
    num_common_col_values = len(common_col_values)
    electri_classic_common_unique_values[col] = num_common_col_values
    
print("Electric Bike vs Classic Bike")
print("--> Número de valores únicos comunes")
for columna, num_valores in electri_classic_common_unique_values.items():
    print(f"----> Columna '{columna}': {num_valores}")

Electric Bike vs Classic Bike
--> Número de valores únicos comunes
----> Columna 'start_lat': 479
----> Columna 'start_lng': 514
----> Columna 'end_lat': 772
----> Columna 'end_lng': 754


**OBSERVACIONES 3.1 (Observaciones Detalladas sobre Coordenadas de Finalización):**

* Contraste en Coordenadas de Finalización: Resulta particularmente interesante observar que, para las bicicletas eléctricas, el número de coordenadas únicas en las columnas de finalización de viaje ('end_lat' y 'end_lng') es significativamente menor que el observado para las coordenadas de inicio. Además, estas coordenadas de finalización presentan una similitud notable con las correspondientes a bicicletas de la categoría 'classic_bike', lo que se traduce a este raro fenómeno para las bicicletas eléctricas: 

* Los usuarios retornan las bicicletas en un número relativamente pequeño de puntos (un poco más de mil doscientos), pero toman las bicicletas de un número muy alto de puntos (más de medio millón).
  
Preguntas para Investigación Futura:

* Preferencias de Estacionamiento de los Usuarios: ¿Existe una tendencia entre los usuarios de bicicletas eléctricas a dejar estas en estaciones específicas para bicicletas eléctricas, posiblemente para evitar cargos adicionales asociados con el estacionamiento fuera de estaciones designadas?
  
* Estrategias de Redistribución por Divvy: ¿Participa Divvy activamente en la redistribución de bicicletas eléctricas, colocándolas en postes y racks públicos o priorizando su retorno a estaciones específicas, posiblemente incentivado por programas como 'bike angels'?

* Calidad de los Datos de Coordenadas: ¿Podría la discrepancia en la cantidad de coordenadas únicas reflejar un problema subyacente en la precisión de los datos o en la metodología de medición, especialmente para las coordenadas de inicio?

# 7. FIN DE LIMPIEZA Y EXPORTACIÓN DE DATAFRAME COMO ARCHIVO 

                              '''

Con los pasos de limpieza y enriquecimiento de datos completados, estamos listos para exportar nuestro DataFrame trips limpio y preparado para análisis futuros. La exportación se realizará en formato Parquet, elegido por su eficiencia en el almacenamiento y la rapidez en la lectura, especialmente adecuado para manejar grandes conjuntos de datos como el nuestro.

--> Exportación como Archivo Parquet:

* El siguiente comando realiza la exportación, guardando el DataFrame trips en un archivo Parquet denominado "trips_2022_clean.parquet":

In [44]:
trips.to_parquet("trips_2022_clean.parquet")

Con esto, hemos completado la limpieza y preparación de nuestro conjunto de datos de viajes en bicicleta para el año 2022. El archivo Parquet resultante está optimizado para análisis eficientes, marcando el fin de una etapa crucial en nuestro proceso de análisis de datos y abriendo la puerta a exploraciones y descubrimientos en los datos de Cyclistic (Divvy).